In [ ]:
#imports
import re
import time
import nltk
import pickle
import difflib
import datetime
import unicodedata
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
from konlpy.tag import Okt; t=Okt()
from konlpy.tag import Hannanum; hannanum = Hannanum()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from soynlp.hangle import levenshtein, jamo_levenshtein

In [ ]:
#콘솔창 넓게 보기
pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 10)

In [ ]:
# 크롤링 후 보관중인 기업 text file 전부 가져오기

# 기업명단
corp_name_list = pd.read_csv('D:/2020/paper/0.text_crawler/corp_names_v2.csv', sep=',') #사업보고서 없는 기업들 제거한 리스트

# html tag 제거용
pattern = re.compile('<.*?>')

# 빈 리스트 775개 생성
content_corp_total = [[] for _ in range(len(corp_name_list['name']))] 
for i in tqdm(range(len(content_corp_total))):
    for j in range(1, 11):
        try:
            content_corp_total[i].append(open('D:/2020/paper/data_folder_new/' + corp_name_list['name'][i] + '_Report/' + corp_name_list['name'][i] + '_content' + str(j) + '.txt', 'r', encoding='UTF-8').readlines())
            content_corp_total[i][j - 1][0] = eval(unicodedata.normalize('NFKD', re.sub(pattern, '', str(str(content_corp_total[i][j - 1][0]).replace(u'\\xa0', u'')))))
        except:
            content_corp_total[i][j - 1][0] = [unicodedata.normalize('NFKD', re.sub(pattern, '',
                                                 str(str(content_corp_total[i][j - 1][0]).replace(u'\\xa0', u'').replace(u'\\', u'')
                                                     .replace(u'[\'[', u'').replace(u']\']', u'').replace(u', ', u'').replace(u'  ', u' '))))]
            pass

In [ ]:
# 1차 전처리 끝난 텍스트 txt 파일로 export
save_data_path = 'D:/2020/paper/content_doc.txt'
with open(save_data_path, 'wb') as fp:
    pickle.dump(content_corp_total, fp)

In [ ]:
# 단어 토큰 형태로 저장된 kospi 기업 사업보고서 파일 불러오기
load_data_path = 'D:/2020/paper/8.text_data_pickled/content_total_noun.pkl'
with open (load_data_path, 'rb') as fp:
    content_total_word = pickle.load(fp)

In [ ]:
# Cosine 유사도 계산
def TFIDF_Cosine_Calc(content_doc):

    import time
    import datetime
    from tqdm import tqdm

    start_time_cos = time.time()

    firm_cosine = [[] for _ in range(len(content_doc))]
    firm_cosine_mat = [[] for _ in range(len(content_doc))]

    # 기업별로 10개씩 TFIDF 유사도 담을 list 생성
    for i in range(len(firm_cosine)):
        firm_cosine[i] = [[] for _ in range(10)]
        firm_cosine_mat[i] = [[] for _ in range(10)]

    for i in tqdm(range(len(content_doc))): # 775개 기업들
        for j in range(len(content_doc[i])): # 보고서 10개 항목
            for k in range(len(content_doc[i][j])):
                firm_cosine[i][j].append(content_doc[i][j][k][0])

    year = datetime.date.today().year
    years = [[] for _ in range(len(content_doc))]
    for i in range(len(content_doc)):
        years[i] = [[] for _ in range(10)]

    for i in range(len(content_doc)):
        for j in range(len(content_doc[i])):
            for k in range(len(content_doc[i][j])):
                years[i][j].append(str((year-1) - k))

    tfidf_vectorizer = TfidfVectorizer(min_df=1)

    for i in tqdm(range(len(content_doc))):
        for j in range(len(content_doc[i])):
            for k in range(len(content_doc[i][j])):
                firm_cosine_mat[i][j] = pd.DataFrame((tfidf_vectorizer.fit_transform(
                    firm_cosine[i][j]) * (tfidf_vectorizer.fit_transform(
                    firm_cosine[i][j])).T).toarray())

    print("TFIDF-Cosine Simil Matrix 계산, " + str(round(time.time() - start_time_cos, 2)) + "초 소요")

    return firm_cosine_mat

In [ ]:
# Jaccard 유사도 계산
def Jaccard_Calc(content_doc_jacc):

    start_time = time.time()

    # 사업보고서 10개 항목 담을 empty nested list 생성
    firm_jacc = [[] for _ in range(len(content_doc_jacc))]

    for i in range(len(firm_jacc)):
        firm_jacc[i] = [[] for _ in range(10)]

    # 사업보고서 각 항목에 대해 list 안에 empty dataframe 생성
    for i in range(len(content_doc_jacc)):
        for j in range(10):
            firm_jacc[i][j] = [
                pd.DataFrame(np.zeros((len(content_doc_jacc[i][j]), len(content_doc_jacc[i][j])))) for _ in range(1)]

    year = datetime.date.today().year
    years = [[] for _ in range(len(content_doc_jacc))]
    for i in range(len(content_doc_jacc)):
        years[i] = [[] for _ in range(10)]

    for i in range(len(content_doc_jacc)):
        for j in range(len(content_doc_jacc[i])):
            for k in range(len(content_doc_jacc[i][j])):
                years[i][j].append(str((year - 1) - k))

    for i in tqdm(range(len(content_doc_jacc))):
        for l in range(len(content_doc_jacc[i])):
            if len(content_doc_jacc[i][l]) >= 4:
                    for j in range(len(content_doc_jacc[i][l])):
                        for k in range(len(content_doc_jacc[i][l])):
                            try:
                                if j == k:
                                    firm_jacc[i][l][0].iloc[j][k] = 1
                                elif j == k - 1:
                                    firm_jacc[i][l][0].iloc[j][k] = len(
                                set((content_doc_jacc[i][l][j][0])) & set((content_doc_jacc[i][l][k][0]))) / len(
                                set((content_doc_jacc[i][l][j][0])) | set((content_doc_jacc[i][l][k][0])))
                                else:
                                    firm_jacc[i][l][0].iloc[j][k] = 0
                            except:
                                pass

    for i in range(len(firm_jacc)):
        for j in range(10):
            firm_jacc[i][j][0].columns = years[i][j]
            firm_jacc[i][j][0].index = years[i][j]

    for i in range(len(firm_jacc)):
        firm_jacc[i] = pd.DataFrame(np.reshape(firm_jacc[i][0], (len(firm_jacc[i][0]),len(firm_jacc[i][0]))), columns=years[i], index=years[i])

    print("Jaccard Simil Matrix 계산, " + str(round(time.time() - start_time,2)) + "초 소요")

    return firm_jacc

In [ ]:
# Simple Diff 유사도 계산 ( 원하는 보고서 항목 번호 입력 하면, 해당 항목에 대해 lag1(전년대비) 유사도만 계산 )
def Diff_calc(content_doc_simdiff, content_num):

    start_time_diff = time.time()

    #사업보고서 10개 항목 담을 empty nested list 생성
    content_corp_indiv = [[] for _ in range(len(content_doc_simdiff))] #775개

    #사업보고서 각 항목에 대해 필요한만큼의 empty list 생성
    for i in range(len(content_corp_indiv)):
        content_corp_indiv[i] = [[] for _ in range(len(content_doc_simdiff[i][content_num])-1)] #각 항목 마다 전체 연도수 - 1 개 만큼의 빈 리스트 삽입

    #사업보고서 항목 순서대로(1~11) 문서의 Diff 계산해서 List Index(0~10)에 append
    for i in tqdm(range(len(content_doc_simdiff))):
        for j in range(len(content_doc_simdiff[i][content_num])):
            try:
                for line in difflib.unified_diff(content_doc_simdiff[i][content_num][j][0], content_doc_simdiff[i][content_num][j + 1][0]):
                    content_corp_indiv[i][j].append(line)
            except:
                pass

    # c 계산하는 방법
    # (content_corp_diff[2][0][1] + content_corp_diff[2][0][2])  / (len(nltk.sent_tokenize(content_corp_docu[2][0][0][0])) + len(nltk.sent_tokenize(content_corp_docu[2][0][1][0])))

    for i in tqdm(range(len(content_corp_indiv))):
        for j in range(len(content_corp_indiv[i])):
            content_corp_indiv[i][j] = pd.DataFrame(content_corp_indiv[i][j], columns=['Text_Diff'])
            content_corp_indiv[i][j]['nonChanges'] = 0
            content_corp_indiv[i][j]['additions'] = 0
            content_corp_indiv[i][j]['deletions'] = 0

    #항목별로 20년치 문서 돌면서 +,-,' ' 표기 체크
    for i in tqdm(range(len(content_corp_indiv))):
        for j in range(len(content_corp_indiv[i])):
            for k in range(len(content_corp_indiv[i][j])):
                if '+' in content_corp_indiv[i][j]['Text_Diff'][k]:
                    content_corp_indiv[i][j]['additions'][k] = 1
                elif '-' in content_corp_indiv[i][j]['Text_Diff'][k]:
                    content_corp_indiv[i][j]['deletions'][k] = 1
                elif (content_corp_indiv[i][j]['additions'][k] and content_corp_indiv[i][j]['deletions'][k]) == 0:
                    content_corp_indiv[i][j]['nonChanges'][k] = 1

    content_corp_diff = [[] for _ in range(len(content_corp_indiv))]
    for i in range(len(content_corp_diff)):
        content_corp_diff[i] = [[] for _ in range(len(content_corp_indiv[i]))]

    for i in tqdm(range(len(content_corp_diff))):
        for j in range(len(content_corp_diff[i])):
            content_corp_diff[i][j] = content_corp_indiv[i][j].sum()[1:]

    # c 계산을 위한, 2개년도 문서별 document size 계산
    docu_diff_size = [[] for _ in range(len(content_doc_simdiff))]

    for i in tqdm(range(len(content_doc_simdiff))):
        for j in range(len(content_doc_simdiff[i][content_num])-1):
            docu_diff_size[i].append(int(len(content_doc_simdiff[i][content_num][j][0]) + len(content_doc_simdiff[i][content_num][j+1][0])))

    # c 계산
    docu_diff_c = [[] for _ in range(len(content_doc_simdiff))]

    for i in range(len(content_doc_simdiff)):
        for j in range(len(content_doc_simdiff[i][content_num])-1):
            if (content_corp_diff[i][j][1] + content_corp_diff[i][j][2]) == 0:
                content_corp_diff[i][j][1] = docu_diff_size[i][j]/2
                content_corp_diff[i][j][2] = docu_diff_size[i][j]/2
            docu_diff_c[i].append(round(float((content_corp_diff[i][j][1] + content_corp_diff[i][j][2])/docu_diff_size[i][j]), 3))

    # sim_simple 계산, def) [Cmax - c] / Cmax
    docu_diff_simple = [[] for _ in range(len(content_doc_simdiff))]

    for i in range(len(content_doc_simdiff)):
        for j in range(len(content_doc_simdiff[i][content_num]) - 1):
            docu_diff_simple[i].append(round(float((max(docu_diff_c[i]) - docu_diff_c[i][j]) / max(docu_diff_c[i])), 3))

    year = datetime.date.today().year
    years = [[] for _ in range(len(content_doc_simdiff))]
    for i in range(len(content_doc_simdiff)):
        for j in range(len(content_doc_simdiff[i][content_num])):
            years[i].append(str((year - 1) - j))

    diff_list = [[] for _ in range(len(content_doc_simdiff))]
    for i in range(len(diff_list)):
        diff_list[i] = pd.DataFrame(columns=years[i], index=years[i], dtype=float)

    for i in tqdm(range(len(diff_list))):
        for j in range(len(diff_list[i])):
            for k in range(len(diff_list[i])):
                if j == k:
                    diff_list[i].iloc[j][k] = 1
                elif j == k - 1 :
                    diff_list[i].iloc[j][k] = docu_diff_simple[i][k-1]
                else :
                    diff_list[i].iloc[j][k] = 0

    print("Simple_Diff 계산 약 " + str(round(time.time() - start_time_diff,2)) + '초 소요')

    year = datetime.date.today().year
    years = [[] for _ in range(len(diff_list))]
    for i in range(len(diff_list)):
        for j in range(len(diff_list[i])):
                years[i].append(str((year - 1) - j))

    year_max = []
    for i in range(len(years)):
        year_max.append(len(years[i]))

    years_max_v2 = [ str( year - 1 - k ) + "~" + str( year - 1 - ( k + 1 ) ) for k in range( max( year_max ) - 1) ]
    lag1_simdiff = pd.DataFrame([], index=corp_name_list['name'], columns=years_max_v2)
    for i in tqdm(range(len(diff_list))):
        if len(diff_list[i]) >= 4 :
            for j in range(0, len(diff_list[i])-1):
                for k in range(0, len(diff_list[i]) - 1):
                    if j == k-1 :
                        lag1_simdiff.iloc[i][j] = diff_list[i].iloc[j][k]

    return lag1_simdiff